In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize


In [ ]:
prices_close = pd.read_csv('sp500_prices_close.csv', index_col=0, parse_dates=True)
prices_close = prices_close.dropna()
print(prices_close.head())


# Engle-Granger test:

Assuming 2 time-series $X_{t}$ and $Y_{t}$ are I(1), they are cointegrated if there exist scalar $\beta$ such that :
 
$u_{t}= Y_{t}​− \beta X_{t}$ satisfies $u_{t}$~I(0)  

In [ ]:
# Differentiation of a time-series
def diff(series, d=1):
    """Return Δ^d series (d-th difference)."""
    out = series.copy()
    for _ in range(d):
        out = out.diff()
    return out.dropna()

In [ ]:
# ADF test
def adf_test(series, max_lags=None, ic='AIC', trend='c'):
    """
    trend: 'n' (no const), 'c' (const), 'ct' (const+trend), 'ctt' (const+trend+quad)
    ic:    'AIC' or 'BIC' for autolag selection
    """
    series = series.dropna()
    res = adfuller(series, maxlag=max_lags, regression=trend, autolag=ic)
    out = {
        'ADF stat': res[0],
        'p-value':  res[1],
        '#lags':    res[2],
        '#obs':     res[3],
        'crit':     res[4],   # dict with 1%, 5%, 10%
        'icbest':   res[5] if len(res) > 5 else None
    }
    return out

In [ ]:
#checks the order of integration of a time-series
def is_Id(series, d=1, adf_trend='c', ic='AIC'):
    """Quick check: not stationary at level but stationary after one diff."""
    r0 = adf_test(series, trend=adf_trend, ic=ic)['p-value']
    r1 = adf_test(diff(series, d=d), trend='n', ic=ic)['p-value']
    return (r0 > 0.05) and (r1 < 0.05)

In [ ]:
print(adf_test(diff(prices_close['AAPL']), trend='c'))

In [ ]:
print(is_Id(prices_close['AAPL'],d=0))
print(is_Id(prices_close['AAPL'],d=1))

In [ ]:
def engle_granger(X,Y, p_val=0.05):
    X, Y = X.align(Y, join="inner")
    if not (is_Id(X,d=1) and is_Id(Y,d=1)):
        return None
    else:
        score, p_value, _ = coint(X, Y)

        if p_value < p_val:
            X_const = sm.add_constant(X)
            model = sm.OLS(Y, X_const).fit()
            beta = model.params[1]  # slope coefficient
            alpha = model.params[0]  # intercept
            
            return beta, alpha, p_value, score
        else:
            return None

In [ ]:
print(engle_granger(prices_close['AAPL'], prices_close['MSFT'], p_val=0.05))

# Finding cointegrated pairs in the SP500

In [ ]:
def cointegrated_pairs(df, p_val=0.05):
    pairs=[]
    for i in range(df.shape[1]):
        for j in range(i+1,df.shape[1]):
            if i!=j:
                    X = df.iloc[:,i]
                    Y = df.iloc[:,j]
                    if not engle_granger(X,Y, p_val=p_val) is None: 
                        beta, alpha, p_value, score = engle_granger(X,Y, p_val=p_val)
                        if p_value < p_val:
                            print(f"Cointegrated pair: {df.columns[i]} and {df.columns[j]} with p-value {p_value}")
                            pairs.append([df.columns[i], df.columns[j], beta, alpha, p_value, score])
    return pairs

In [ ]:
## Took too long to run
# pairs = cointegrated_pairs(returns_close, p_val=0.01)
pairs=  cointegrated_pairs(prices_close, p_val=0.05)
print(pairs)

In [ ]:
pair_test=[['AAPL', 'MSFT', 1.057, -3.123, 0.0001, -4.321],['GOOGL', 'AMZN', 0.876, 2.456, 0.00005, -4.567],['FB', 'TWTR', 1.234, -1.234, 0.0002, -4.123],['NFLX', 'DIS', 0.987, 0.567, 0.00015, -4.234],['AAPL', 'GOOGL', 1.345, -2.345, 0.0003, -4.456]]



# Finding the tickers that represent the whole index

In [ ]:
def minimum_pairs(pairs):
    Min=[]
    Min.append(pairs[0][0])
    Min.append(pairs[0][1])
    pairs.pop(0)
    while len(pairs)>0:
        if pairs[0][0] in Min and pairs[0][1] in Min:
            pairs.pop(0)
        else:
            if pairs[0][0] not in Min:
                Min.append(pairs[0][0])
            if pairs[0][1] not in Min:
                Min.append(pairs[0][1])
            pairs.pop(0)
    return Min, len(Min)

In [ ]:
print(minimum_pairs(pair_test))

# Building investing strategies (Portfolio Building)

In [ ]:
def buy_and_hold(df,ticker,capital=100):
    price=[]
    for i in range(len(df)):
        price.append(df[ticker].iloc[i]*capital)
    return price

In [ ]:
plt.plot(buy_and_hold(prices_close,'AAPL',capital=1000))
plt.plot(buy_and_hold(prices_close,'MSFT',capital=1000))
plt.plot(buy_and_hold(prices_close,'NVDA',capital=1000))
plt.title('Buy and Hold Strategy - start date is 2010-01-04 and end date is 2023-10-31')
plt.legend(['AAPL','MSFT','NVDA'])
plt.show()

In [ ]:
def portfolio(df,tickers,capitals):
    port=[]
    for i in range(len(df)):
        total=0
        for j in range(len(tickers)):
            total+=df[tickers[j]].iloc[i]*capitals[j]
        port.append(total)
    return port

In [ ]:
bigtech=portfolio(prices_close,['AAPL','MSFT','NVDA'],[0.4,0.4,0.2])
plt.plot(bigtech)
plt.title('Big Tech Portfolio - start date is 2010-01-04 and end date is 2023-10-31')
plt.legend(['Big Tech Portfolio'])
plt.show()

In [ ]:
def best_portfolio(df,tickers): ## finding the best portfolio with the highest return using function optimization
    
    def herfindahl_hirschman_index(weights):
        """Calculate the Herfindahl-Hirschman Index (HHI) for a given set of weights."""
        return sum(w**2 for w in weights)
    ## Adding diversification constraint (Herfindahl-Hirschman Index)
    def objective_function(weights):
        port = portfolio(df, tickers, weights)
        return -((port[-1]-port[0])/port[0]) + 0.5 * herfindahl_hirschman_index(weights)  # Minimize negative return

    constraints = ({'type': 'eq', 'fun': lambda x: sum(x) - 1})  # Weights must sum to 1
    bounds = tuple((0, 1) for _ in tickers)  # Weights must be between 0 and 1
    initial_weights = [1/len(tickers)] * len(tickers)

    result = minimize(objective_function, initial_weights, bounds=bounds, constraints=constraints)

    return result.x, -result.fun  # Return optimal weights and maximum return


In [ ]:
best_weights, _ = best_portfolio(prices_close,['MSFT','GOOGL','AMZN'])

In [ ]:
plt.plot(bigtech)
plt.plot(portfolio(prices_close,['MSFT','GOOGL','AMZN'],best_weights))
plt.legend(['Big Tech Portfolio', 'Optimal Portfolio'])
plt.show()

In [ ]:
def cointegrated_strat(df,pairs, initial_capital=10000):
    capital=initial_capital
    for pair in pairs:
        ticker1=pair[0]
        ticker2=pair[1]
        beta=pair[2]
        alpha=pair[3]
        price1=buy_and_hold(df,ticker1,capital=initial_capital/2)
        price2=buy_and_hold(df,ticker2,capital=initial_capital/2)

        # Calculate the spread
        spread = np.array(price2) - beta * np.array(price1) - alpha
        
        # Calculate the mean and standard deviation of the spread
        spread_mean = np.mean(spread)
        spread_std = np.std(spread)
        
        # Set entry and exit thresholds
        entry_threshold = spread_mean + spread_std
        exit_threshold = spread_mean
        
        position1 = 0  # Position in ticker1
        position2 = 0  # Position in ticker2
        
        for t in range(len(spread)):
            if spread[t] > entry_threshold and position1 == 0 and position2 == 0:
                # Short ticker2 and long ticker1
                position1 = capital / price1[t]  # Buy as many shares of ticker1 as possible
                position2 = -capital / price2[t]  # Short as many shares of ticker2 as possible
                capital = 0  # All capital is now invested
            
            elif spread[t] < exit_threshold and position1 != 0 and position2 != 0:
                # Close positions
                capital += position1 * price1[t] + position2 * price2[t]
                position1 = 0
                position2 = 0
        
        # At the end of the period, close any open positions
        if position1 != 0 and position2 != 0:
            capital += position1 * price1[-1] + position2 * price2[-1]
    
    return capital